In [1]:
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.eps'
if not 'location' in locals():
    #save location.  First one is for running on home PC, second for running on the work laptop.  May need to make a global change
    location = 'E:\\Documents\\Dan\\Code\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'
    #location = 'C:\\Users\\dhendrickson\\Documents\\github\\FigsAndPlots\\FigsAndPlotsDocument\\Figures\\'

my_cmap = plt.get_cmap('gray')
#Standard cycle for collors and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '--', ':', '-.']))
plt.rc('axes', prop_cycle=default_cycler)

In [2]:
#Project Specific packages:
import zipfile
#import DWFT as fp
import os as os
import pandas as pd
import random
import multiprocessing
from joblib import Parallel, delayed
import tictoc as tt

import pywt
from pywt._extensions._pywt import (DiscreteContinuousWavelet, ContinuousWavelet,
                                Wavelet, _check_dtype)
from pywt._functions import integrate_wavelet, scale2frequency
from time import time as ti

In [3]:
#DataSet = np.genfromtxt(open("E:\\Documents\\Dan\\PhD\\Play\\ASC\\60kPoints-210709-1026.csv",'r'), delimiter=',',skip_header=4)
#DataSet = np.genfromtxt(open("C:\\Users\\dhendrickson\\Pone Drive\\OneDrive - The Port of Virginia\\Shared with Everyone\\60kAccel-210713-1700.csv",'r'), delimiter=',',skip_header=4)
#folder = "C:\\Users\\hendrickson\\Desktop\\temp\\"
#folder = "g:\\"
folder = "g:\\Excel Versions\\"


In [4]:
def cwt_fixed(data, scales, wavelet, sampling_period=1.):
    """
    COPIED AND FIXED FROM pywt.cwt TO BE ABLE TO USE WAVELET FAMILIES SUCH
    AS COIF AND DB

    COPIED From Spenser Kirn
    
    All wavelet work except bior family, rbio family, haar, and db1.
    
    cwt(data, scales, wavelet)

    One dimensional Continuous Wavelet Transform.

    Parameters
    ----------
    data : array_like
        Input signal
    scales : array_like
        scales to use
    wavelet : Wavelet object or name
        Wavelet to use
    sampling_period : float
        Sampling period for frequencies output (optional)

    Returns
    -------
    coefs : array_like
        Continous wavelet transform of the input signal for the given scales
        and wavelet
    frequencies : array_like
        if the unit of sampling period are seconds and given, than frequencies
        are in hertz. Otherwise Sampling period of 1 is assumed.

    Notes
    -----
    Size of coefficients arrays depends on the length of the input array and
    the length of given scales.

    Examples
    --------
    >>> import pywt
    >>> import numpy as np
    >>> import matplotlib.pyplot as plt
    >>> x = np.arange(512)
    >>> y = np.sin(2*np.pi*x/32)
    >>> coef, freqs=pywt.cwt(y,np.arange(1,129),'gaus1')
    >>> plt.matshow(coef) # doctest: +SKIP
    >>> plt.show() # doctest: +SKIP
    ----------
    >>> import pywt
    >>> import numpy as np
    >>> import matplotlib.pyplot as plt
    >>> t = np.linspace(-1, 1, 200, endpoint=False)
    >>> sig  = np.cos(2 * np.pi * 7 * t) + np.real(np.exp(-7*(t-0.4)**2)*np.exp(1j*2*np.pi*2*(t-0.4)))
    >>> widths = np.arange(1, 31)
    >>> cwtmatr, freqs = pywt.cwt(sig, widths, 'mexh')
    >>> plt.imshow(cwtmatr, extent=[-1, 1, 1, 31], cmap='PRGn', aspect='auto',
    ...            vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max())  # doctest: +SKIP
    >>> plt.show() # doctest: +SKIP
    """

    # accept array_like input; make a copy to ensure a contiguous array
    dt = _check_dtype(data)
    data = np.array(data, dtype=dt)
    if not isinstance(wavelet, (ContinuousWavelet, Wavelet)):
        wavelet = DiscreteContinuousWavelet(wavelet)
    if np.isscalar(scales):
        scales = np.array([scales])
    if data.ndim == 1:
        try:
            if wavelet.complex_cwt:
                out = np.zeros((np.size(scales), data.size), dtype=complex)
            else:
                out = np.zeros((np.size(scales), data.size))
        except AttributeError:
            out = np.zeros((np.size(scales), data.size))
        for i in np.arange(np.size(scales)):
            precision = 10
            int_psi, x = integrate_wavelet(wavelet, precision=precision)
            step = x[1] - x[0]
            j = np.floor(
                np.arange(scales[i] * (x[-1] - x[0]) + 1) / (scales[i] * step))
            if np.max(j) >= np.size(int_psi):
                j = np.delete(j, np.where((j >= np.size(int_psi)))[0])
            coef = - np.sqrt(scales[i]) * np.diff(
                np.convolve(data, int_psi[j.astype(np.int)][::-1]))
            d = (coef.size - data.size) / 2.
            out[i, :] = coef[int(np.floor(d)):int(-np.ceil(d))]
        frequencies = scale2frequency(wavelet, scales, precision)
        if np.isscalar(frequencies):
            frequencies = np.array([frequencies])
        for i in np.arange(len(frequencies)):
            frequencies[i] /= sampling_period
        return out, frequencies
    else:
        raise ValueError("Only dim == 1 supported")

In [5]:
def getThumbprint(data, wvt, ns=50, numslices=5, slicethickness=0.12, 
                  valleysorpeaks='both', normconstant=1, plot=True):
    '''
    STarted with Spenser Kirn's code, modifed by DCH
    Updated version of the DWFT function above that allows plotting of just
    valleys or just peaks or both. To plot just valleys set valleysorpeaks='valleys'
    to plot just peaks set valleysorpeaks='peaks' or 'both' to plot both.
    '''
    # First take the wavelet transform and then normalize to one
    cfX, freqs = cwt_fixed(data, np.arange(1,ns+1), wvt)
    cfX = np.true_divide(cfX, abs(cfX).max()*normconstant)
    
    fp = np.zeros((len(data), ns), dtype=int)
    
    # Create the list of locations between -1 and 1 to preform slices. Valley
    # slices will all be below 0 and peak slices will all be above 0.
    if valleysorpeaks == 'both':
        slicelocations1 = np.arange(-1 ,0.0/numslices, 1.0/numslices)
        slicelocations2 = np.arange(1.0/numslices, 1+1.0/numslices, 1.0/numslices)
        slicelocations = np.array(np.append(slicelocations1,slicelocations2))
        
    for loc in slicelocations:
        for y in range(0, ns):
            for x in range(0, len(data)):
                if cfX[y, x]>=(loc-(slicethickness/2)) and cfX[y,x]<= (loc+(slicethickness/2)):
                    fp[x,y] = 1
                    
    fp = np.transpose(fp[:,:ns])
    return fp

In [6]:
def RidgeCount(fingerprint):
    '''
    From Spencer Kirn
    Count the number of times the fingerprint changes from 0 to 1 or 1 to 0 in 
    consective rows. Gives a vector representation of the DWFT
    '''
    diff = np.zeros((fingerprint.shape))
    
    for i, row in enumerate(fingerprint):
        if i==0:
            prev = row
        else:
            # First row (i=0) of diff will always be 0s because it does not
            # matter what values are present. 
            # First find where the rows differ
            diff_vec = abs(row-prev)
            # Then set those differences to 1 to be added later
            diff_vec[diff_vec != 0] = 1
            diff[i, :] = diff_vec
            
            prev = row
            
    ridgeCount = diff.sum(axis=0)
    
    return ridgeCount

In [7]:
def getAcceleration(FileName):
    try:
        DataSet = np.genfromtxt(open(folder+FileName,'r'), delimiter=',',skip_header=4)
        return [[FileName,'x',DataSet[:,2]],[FileName,'y',DataSet[:,3]],[FileName,'z',DataSet[:,4]]]
    except:
        return [False,FileName,False]

def makePrints(DataArray):
    FingerPrint = getThumbprint(DataArray[2],'gaus2')
    return [DataArray[0],DataArray[1],FingerPrint]

def getResults(FPnMd):
    Ridges = RidgeCount(FPnMd[2][:,500:59500])
    return [FPnMd[0],FPnMd[1],Ridges]

def CountAboveThreshold(Ridges, Threshold = 10):
    Cnum = np.count_nonzero(Ridges[2] >= Threshold)
    return [Ridges[0],Ridges[1],Cnum]


In [8]:
num_cores = multiprocessing.cpu_count()
files = os.listdir(folder)
GroupSize = 250
loops = int(float(np.size(files))/float(GroupSize))+1


if __name__ == "__main__":
    for i in range(loops):
        AllAccels = Parallel(n_jobs=num_cores)(delayed(getAcceleration)(file) for file in files[i*GroupSize:((i+1)*GroupSize)])
        Flattened = []
        for j in range(np.shape(AllAccels)[0]):
            if AllAccels[j][0] == False:
                print(j,AllAccels[j][1])
            else: 
                for k in range(3):
                    Flattened.append(AllAccels[j][k])
        AllFingers =  Parallel(n_jobs=num_cores)(delayed(makePrints)(datas) for datas in Flattened)
        AllRidges = Parallel(n_jobs=num_cores)(delayed(getResults)(datas) for datas in AllFingers)
        Events=[]
        Events = Parallel(n_jobs=num_cores)(delayed(CountAboveThreshold)(datas) for datas in AllRidges)
        
        Events = np.matrix(Events)
        df = pd.DataFrame(data=Events)
        df.to_csv('G:\\Results\\Random Check' + str(i) + '.csv', sep=',', index = False, header=False,quotechar='"')
        print(str(i+1)+' of '+str(loops))

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


1 of 36
2 of 36
3 of 36
4 of 36
5 of 36
6 of 36
7 of 36
110 60kAccel-210727-1649.csv
125 60kAccel-210727-1910.csv
137 60kAccel-210727-2103.csv
140 60kAccel-210727-2131.csv
162 60kAccel-210728-0057.csv
167 60kAccel-210728-0144.csv
169 60kAccel-210728-0203.csv
181 60kAccel-210728-0355.csv
183 60kAccel-210728-0414.csv
198 60kAccel-210728-0634.csv
201 60kAccel-210728-0701.csv
208 60kAccel-210728-0807.csv
224 60kAccel-210728-1035.csv
230 60kAccel-210728-1131.csv
237 60kAccel-210728-1236.csv
244 60kAccel-210728-1341.csv
246 60kAccel-210728-1400.csv
248 60kAccel-210728-1418.csv
8 of 36
9 60kAccel-210728-1600.csv
14 60kAccel-210728-1647.csv
18 60kAccel-210728-1724.csv
19 60kAccel-210728-1733.csv
32 60kAccel-210728-1933.csv
41 60kAccel-210728-2057.csv
53 60kAccel-210728-2248.csv
62 60kAccel-210729-0012.csv
77 60kAccel-210729-0231.csv
82 60kAccel-210729-0317.csv
86 60kAccel-210729-0354.csv
91 60kAccel-210729-0441.csv
104 60kAccel-210729-0641.csv
105 60kAccel-210729-0651.csv
110 60kAccel-210729-0